Robot Localization with Python and Particle Filters
===================================================

Import libraries and load map.

In [1]:
import numpy as np
import cv2

map = cv2.imread("map.png", 0)
print(map)
HEIGHT, WIDTH = map.shape

rx, ry, rtheta = (WIDTH/4, HEIGHT/4, 0)

[[ 79  80  82 ... 133 148 156]
 [ 78  79  81 ... 138 156 164]
 [ 76  77  79 ... 147 170 180]
 ...
 [181 181 182 ... 174 172 171]
 [178 178 178 ... 180 179 178]
 [177 177 177 ... 183 182 182]]


Map coordinate system

![title](images/coords.png)

CAUTION: The terrain height at X,Y coordinates is map(Y,X).

Read keyboard input.

In [2]:
STEP = 5
TURN=  np.radians(25)
def get_input():
    fwd = 0
    turn = 0
    halt = False
    k = cv2.waitKey(0)
    print(k)
    if k == 56:
        fwd = STEP
    elif k == 54:
        turn = TURN
    elif k == 52:
        turn = -TURN
    else:
        halt = True
    return fwd, turn, halt

Move the robot, with Gausssian noise.

![title](images/gaussian.png)

In [3]:
SIGMA_STEP = 0.5 #pixels
SIGMA_TURN = np.radians(5)

def move_robot(rx, ry, rtheta, fwd, turn):
    fwd_noisy = np.random.normal(fwd, SIGMA_STEP, 1)
    print("fwd_noisy = ", fwd_noisy)
    
    rx += fwd_noisy * np.cos(rtheta)
    ry += fwd_noisy * np.sin(rtheta)
    
    turn_noisy = np.random.normal(turn, SIGMA_TURN, 1)
    print("turn_noisy = ", np.degrees(turn_noisy))
    
    rtheta += turn_noisy

    return rx, ry, rtheta

Initialize particle cloud.

In [4]:
NUM_PARTICLES = 3000

def init():
    particles = np.random.rand(NUM_PARTICLES, 3) #3000 particles with three attributes rx, ry and rtheta
    particles *= np.array( (WIDTH, HEIGHT, np.radians(360)))
    return particles

Move the particles.

In [5]:
def move_particles(particles, fwd, turn):
    particles[:, 0] += fwd * np.cos(particles[:,2])
    particles[:, 1] += fwd * np.sin(particles[:,2])
    particles[:, 2] += turn
    
    particles[:,0] = np.clip(particles[:,0], 0.0, WIDTH-1)
    particles[:,1] = np.clip(particles[:,1], 0.0, HEIGHT-1)
    return particles

Get value from robot's sensor.

In [6]:
SIGMA_SENSOR = 2
def sense(x, y, noisy=False):
    x = int(x)
    y = int(y)
    if noisy:
        return np.random.normal(map[y, x], SIGMA_SENSOR, 1)
    return map[y, x]

Compute particle weights.

In [7]:
def compute_weights(particles, robot_sensor):    
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        elevation = sense(particles[i, 0], particles[i, 1], noisy=False)
        errors[i] = abs(robot_sensor - elevation)
        
    weights = np.max(errors) - errors
    
    weights[
        (particles[:, 0] == 0) |
        (particles[:,0] == WIDTH-1) |
        (particles[:,1] == 0) |
        (particles[:,1] == HEIGHT-1)
    ] = 0.0
    
    weights = weights ** 3
    return weights

Resample the particles.

In [8]:
def resample(particles, weights):
    probabilities = weights/np.sum(weights)
    
    new_index = np.random.choice(
        NUM_PARTICLES,
        size = NUM_PARTICLES,
        p = probabilities)
    
    particles = particles[new_index, :]
     
    return particles

Add noise to the particles.

In [9]:
SIGMA_POS = 2
SIGMA_TURN = np.radians(10)

def add_noise(particles):
    noise  = np.concatenate(
        (
            np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
            np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
            np.random.normal(0, SIGMA_TURN, (NUM_PARTICLES, 1))
        ), 
        axis=1
    )
    particles += noise
    return particles

Display robot, particles and best guess.

In [10]:
def display(map, rx, ry, particles):
    lmap = cv2.cvtColor(map, cv2.COLOR_GRAY2BGR)
    
    # Display particles
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            cv2.circle(lmap, 
                       (int(particles[i,0]), int(particles[i,1])), 
                       1, 
                       (255,0,0), 
                       1)
        
    # Display robot
    cv2.circle(lmap, (int(rx), int(ry)), 5, (0,255,0), 10)

    # Display best guess
    if len(particles) > 0:
        px = np.mean(particles[:,0])
        py = np.mean(particles[:,1])
        cv2.circle(lmap, (int(px), int(py)), 5, (0,0,255), 5)

    cv2.imshow('map', lmap)

Main routine.

In [11]:
particles = init()
while True:
    display(map, rx, ry, particles)
    fwd, turn, halt = get_input()
    if halt:
        break
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)        
        weights = compute_weights(particles, robot_sensor)
        particles = resample(particles, weights)
        particles = add_noise(particles)
    cv2.destroyAllWindows()                        

cv2.destroyAllWindows()                        


56
fwd_noisy =  [4.2688326]
turn_noisy =  [-4.10421229]
56
fwd_noisy =  [4.59537533]
turn_noisy =  [-6.02328109]
54
fwd_noisy =  [-0.6825363]
turn_noisy =  [32.98204465]
56
fwd_noisy =  [5.37425834]
turn_noisy =  [-17.12754428]
56
fwd_noisy =  [5.85253304]
turn_noisy =  [-5.20857138]
52
fwd_noisy =  [0.1930701]
turn_noisy =  [-24.59695729]
56
fwd_noisy =  [4.54994713]
turn_noisy =  [5.30424001]
56
fwd_noisy =  [4.82378856]
turn_noisy =  [10.11350175]
56
fwd_noisy =  [4.91326639]
turn_noisy =  [-8.49930604]
52
fwd_noisy =  [-0.38106633]
turn_noisy =  [-8.42536451]
56
fwd_noisy =  [4.46894252]
turn_noisy =  [5.68176454]
56
fwd_noisy =  [4.90256997]
turn_noisy =  [-9.34060703]
56
fwd_noisy =  [4.60830931]
turn_noisy =  [23.42068741]
56
fwd_noisy =  [4.92160932]
turn_noisy =  [9.71181692]
56
fwd_noisy =  [4.79201217]
turn_noisy =  [-8.40665728]
54
fwd_noisy =  [-0.69510734]
turn_noisy =  [32.47772844]
54
fwd_noisy =  [0.54101781]
turn_noisy =  [35.20013168]
56
fwd_noisy =  [5.61644974]
tur

56
fwd_noisy =  [5.39352181]
turn_noisy =  [6.9112838]
56
fwd_noisy =  [5.0237907]
turn_noisy =  [-11.67577913]
56
fwd_noisy =  [5.2350851]
turn_noisy =  [-9.72926487]
56
fwd_noisy =  [4.80574436]
turn_noisy =  [-16.18317357]
56
fwd_noisy =  [5.23295397]
turn_noisy =  [-0.65562204]
56
fwd_noisy =  [5.26131121]
turn_noisy =  [-0.0238386]
56
fwd_noisy =  [5.39931929]
turn_noisy =  [14.22203661]
52
fwd_noisy =  [-0.25889195]
turn_noisy =  [-32.44070039]
56
fwd_noisy =  [4.85844605]
turn_noisy =  [-3.1333771]
56
fwd_noisy =  [4.47281187]
turn_noisy =  [9.52387674]
56
fwd_noisy =  [4.19718652]
turn_noisy =  [1.68616504]
56
fwd_noisy =  [4.4843228]
turn_noisy =  [3.48994625]
52
fwd_noisy =  [-0.33599221]
turn_noisy =  [-25.52122346]
56
fwd_noisy =  [4.70807214]
turn_noisy =  [9.07540221]
52
fwd_noisy =  [-0.47609594]
turn_noisy =  [-42.58517937]
56
fwd_noisy =  [4.96576062]
turn_noisy =  [4.92542936]
52
fwd_noisy =  [0.27355384]
turn_noisy =  [-26.77953221]
56
fwd_noisy =  [5.40862235]
turn_